In [365]:
import torch
import torch.optim as optim
torch.manual_seed(777)

#데이터 가져오기
import numpy as np 
import pandas as pd

train_data=pd.read_csv('train.csv')
train_data

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.000000
1,0,0,30,0,0,1.5,69.06,-12,0.000000
2,0,1,0,0,0,1.6,71.78,-12,0.000000
3,0,1,30,0,0,1.6,71.75,-12,0.000000
4,0,2,0,0,0,1.6,75.20,-12,0.000000
...,...,...,...,...,...,...,...,...,...
9994,208,5,0,0,0,0.5,64.12,12,0.000000
9995,208,5,30,26,383,0.8,56.18,13,5.630068
9996,208,6,0,41,578,1.1,47.46,15,13.887196
9997,208,6,30,52,699,1.4,44.51,17,23.269925


In [366]:
x_train=train_data.copy()
x_train=x_train.drop('TARGET',axis=1)


y_train=train_data['TARGET'].copy()

In [367]:
x_train

,Day,Hour,Minute,DHI,DNI,WS,RH,T
0,0,0,0,0,0,1.5,69.08,-12
1,0,0,30,0,0,1.5,69.06,-12
2,0,1,0,0,0,1.6,71.78,-12
3,0,1,30,0,0,1.6,71.75,-12
4,0,2,0,0,0,1.6,75.20,-12
...,...,...,...,...,...,...,...,...
9994,208,5,0,0,0,0.5,64.12,12
9995,208,5,30,26,383,0.8,56.18,13
9996,208,6,0,41,578,1.1,47.46,15
9997,208,6,30,52,699,1.4,44.51,17


In [368]:
y_train

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
          ...    
9994     0.000000
9995     5.630068
9996    13.887196
9997    23.269925
9998    33.027555
Name: TARGET, Length: 9999, dtype: float64

In [369]:
#데이터 표준화
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

x_train=scaler.fit_transform(x_train)

In [370]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train).reshape(9999,1)

print(x_train)
print(y_train)

tensor([[-1.7237, -1.6589, -0.9999,  ..., -0.6411,  0.2243, -2.1140],
        [-1.7237, -1.6589,  1.0001,  ..., -0.6411,  0.2234, -2.1140],
        [-1.7237, -1.5145, -0.9999,  ..., -0.5616,  0.3471, -2.1140],
        ...,
        [ 1.7352, -0.7925, -0.9999,  ..., -0.9593, -0.7593,  0.7033],
        [ 1.7352, -0.7925,  1.0001,  ..., -0.7207, -0.8935,  0.9120],
        [ 1.7352, -0.6481, -0.9999,  ..., -0.4820, -1.1987,  1.1207]])
tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        ...,
        [13.8872],
        [23.2699],
        [33.0276]])


In [371]:
## 모델 설계
linear1 = torch.nn.Linear(8, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 256, bias=True)
linear4 = torch.nn.Linear(256, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4)
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for stop in range(1000):
    
    # 그래디언트 초기화
    optimizer.zero_grad()
    # Forward 계산
    hypothesis = model(x_train)
    # Error 계산
    cost = loss(hypothesis, y_train)
    # Backward 계산 
    cost.backward()
    # 가중치 갱신
    optimizer.step()

    if stop % 100 == 0:
        print(stop, cost.item())

0 1031.8739013671875
100 682.176513671875
200 662.8751831054688
300 412.6672058105469
400 64.11743927001953
500 41.760353088378906
600 32.92384719848633
700 28.262956619262695
800 25.2273006439209
900 22.757007598876953


In [372]:
## w,b 평가

with torch.no_grad(): # 임시로 required_grad = false로 설정하는 것과 같다.
    predicted = model(x_train)
    print('\n Predict: ', predicted)


 Predict:  tensor([[ 0.2378],
        [ 0.2828],
        [ 0.2084],
        ...,
        [21.8469],
        [34.3077],
        [45.0418]])


In [373]:
#테스트 데이터 가져오기
test_data=pd.read_csv('X_test.csv')
test_data=test_data.drop('ID',axis=1)
test_data

,Day,Hour,Minute,DHI,DNI,WS,RH,T
0,1043,7,30,34,584,2.0,63.02,3
1,1043,8,0,45,730,2.3,61.54,5
2,1043,8,30,52,817,2.4,57.34,6
3,1043,9,0,57,878,2.6,52.46,8
4,1043,9,30,61,917,2.2,45.74,10
...,...,...,...,...,...,...,...,...
2476,1094,21,30,0,0,2.4,70.70,-4
2477,1094,22,0,0,0,2.4,66.79,-4
2478,1094,22,30,0,0,2.2,66.78,-4
2479,1094,23,0,0,0,2.1,67.72,-4


In [374]:
test_data=scaler.transform(test_data)
test_data=torch.FloatTensor(test_data)

In [375]:
with torch.no_grad(): # 임시로 required_grad = false로 설정하는 것과 같다.
    predict=model(test_data)

In [376]:
submit=pd.read_csv("sample_submit.csv")

for i, value in enumerate(predict):
    submit["TARGET"][i]=value.item()

submit.to_csv("submit_new_feature_2.csv",index=False)